# Ch `03`: Concept `03`

## Regularization

Import the relevant libraries and initialize the hyper-parameters

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(100)

learning_rate = 0.001
training_epochs = 1000
reg_lambda = 0.

create a helper method to split the dataset

In [2]:
def split_dataset(x_dataset, y_dataset, ratio):
    arr = np.arange(x_dataset.size)
    np.random.shuffle(arr)
    num_train = int(ratio * x_dataset.size)
    x_train = x_dataset[arr[0:num_train]]
    y_train = y_dataset[arr[0:num_train]]
    x_test = x_dataset[arr[num_train:x_dataset.size]]
    y_test = y_dataset[arr[num_train:x_dataset.size]]
    return x_train, x_test, y_train, y_test

Create a fake dataset. y = x^2

In [3]:
x_dataset = np.linspace(-1, 1, 100)

num_coeffs = 9
y_dataset_params = [0.] * num_coeffs
y_dataset_params[2] = 1

print(y_dataset_params)

y_dataset = 0
for i in range(num_coeffs):
    y_dataset += y_dataset_params[i] * np.power(x_dataset, i)
    
y_dataset += np.random.randn(*x_dataset.shape) * 0.3
print(y_dataset)

[0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[ 0.47507036  1.0628082   1.26673515  0.80673016  1.13931002  0.96244849
  0.83862204  0.41615668  0.64603871  0.74592192  0.49936366  0.73548732
  0.39884251  0.78877415  0.71615151  0.45444342  0.29863038  0.73999827
  0.27351799  0.04415966  0.84086234  0.79397834  0.23307823  0.03387269
  0.32073669  0.52609966  0.44468527  0.61507841  0.0907828   0.18821592
  0.22190813 -0.29328547 -0.10191845  0.35604731  0.32318465 -0.05097651
  0.43126685 -0.44341604 -0.35294563 -0.32473495 -0.12649877 -0.17096473
  0.02515121 -0.16663848  0.4022701  -0.51166422 -0.28999354  0.10980309
 -0.48315528  0.44131619 -0.35630325 -0.16400558 -0.27946309 -0.24338022
  0.0409235   0.16468856 -0.24142507  0.39779776  0.00560341 -0.23008647
 -0.21954411  0.05956577  0.13512239  0.07844473 -0.40485125 -0.21521174
  0.29502278  0.34584881  0.44775606 -0.27446894 -0.38084338  0.29848219
  0.10707843  0.01861977  0.85535727  0.10016676  0.51173941 -0.08345573
  0.505

Split the dataset into 70% training and testing 30%

In [4]:
(x_train, x_test, y_train, y_test) = split_dataset(x_dataset, y_dataset, 0.7)

Set up the input/output placeholders

In [5]:
X = tf.placeholder("float")
Y = tf.placeholder("float")

Define our model

In [6]:
def model(X, w):
    terms = []
    for i in range(num_coeffs):
        term = tf.multiply(w[i], tf.pow(X, i))
        terms.append(term)
    return tf.add_n(terms)

Define the regularized cost function

In [7]:
w = tf.Variable([0.] * num_coeffs, name="parameters")
y_model = model(X, w)

cost = tf.div(tf.add(tf.reduce_sum(tf.square(Y-y_model)),                    # loss
                     tf.multiply(reg_lambda, tf.reduce_sum(tf.square(w)))),  # regulation
              2*x_train.size)

train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Set up the session

In [8]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

Try out various regularization parameters 

In [9]:
for reg_lambda in np.linspace(0,1,51):
    for epoch in range(training_epochs):
        sess.run(train_op, feed_dict={X: x_train, Y: y_train})
    final_cost = sess.run(cost, feed_dict={X: x_test, Y:y_test})
    print('reg lambda : {:4.2f} --- final cost : {:8.6f}'.format(reg_lambda, final_cost))

reg lambda : 0.00 --- final cost : 0.032031
reg lambda : 0.02 --- final cost : 0.028902
reg lambda : 0.04 --- final cost : 0.027129
reg lambda : 0.06 --- final cost : 0.025800
reg lambda : 0.08 --- final cost : 0.024813
reg lambda : 0.10 --- final cost : 0.024077
reg lambda : 0.12 --- final cost : 0.023520
reg lambda : 0.14 --- final cost : 0.023092
reg lambda : 0.16 --- final cost : 0.022755
reg lambda : 0.18 --- final cost : 0.022486
reg lambda : 0.20 --- final cost : 0.022267
reg lambda : 0.22 --- final cost : 0.022085
reg lambda : 0.24 --- final cost : 0.021932
reg lambda : 0.26 --- final cost : 0.021801
reg lambda : 0.28 --- final cost : 0.021688
reg lambda : 0.30 --- final cost : 0.021588
reg lambda : 0.32 --- final cost : 0.021500
reg lambda : 0.34 --- final cost : 0.021420
reg lambda : 0.36 --- final cost : 0.021348
reg lambda : 0.38 --- final cost : 0.021282
reg lambda : 0.40 --- final cost : 0.021222
reg lambda : 0.42 --- final cost : 0.021165
reg lambda : 0.44 --- final cost

Close the session

In [10]:
sess.close()